# Load Libraries

In [1]:
import re
import pandas as pd
import numpy as np
import spacy
# import textacy

In [2]:
train = pd.read_csv('train_E6oV3lV.csv')

In [4]:
train_copy = train.copy()

# Load nlp

In [12]:
nlp = spacy.load('en_core_web_md')

In [13]:
from spacy.lang.en import STOP_WORDS

In [9]:
import string

def spacy_tokenizer(sent):
    mytoken = nlp(sent)
    mytoken = [
        word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_
        for word in mytoken
    ]
    mytoken = [word for word in mytoken if word not in STOP_WORDS and word not in string.punctuation]
    return mytoken


In [61]:
re.findall(r'@\w+',train_copy.tweet[6])

['@user', '@user', '@user', '@user', '@user', '@user', '@user', '@user']

In [10]:
train.tweet[0]

' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [11]:
# remove username
train_copy['cleaned'] = train_copy.tweet.apply(lambda x : re.sub(r'@\w+','',x))

In [12]:
train_copy.cleaned[0]

'  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [13]:
def no_of_hashtags(sent):
    return len(re.findall('#\S+',sent))

In [14]:
#count no of hashtags in tweets
train_copy['no_of_hashtags'] = train_copy.cleaned.apply(no_of_hashtags)

In [15]:
from unicodedata import normalize

normalize('NFKD', train_copy.tweet[26]).encode('ascii','ignore').decode("utf-8")


' @user all #smiles when #media is   !!  #pressconference in #antalya #turkey ! sunday #throwback  love! ai  '

In [16]:
# replace unicode strings
from unicodedata import normalize

def unicode_normalize(sent):
    return normalize('NFKD', sent).encode('ascii','ignore').decode("utf-8")

In [17]:
train_copy.cleaned = train_copy.cleaned.apply(unicode_normalize)

# Building function to do the preprocessing all together

In [18]:
def clean(df):
    df['cleaned'] = df.tweet.apply(lambda x : re.sub(r'@\w+','',x))
    df['cleaned'] = df.cleaned.apply(lambda x : normalize('NFKD', x).encode('ascii','ignore').decode("utf-8"))
#     df['no_of_hashtags'] = df.tweet.apply(lambda x : len(re.findall('#\S+',x)))
    
    return df

In [19]:
data = clean(train)

In [20]:
data.head()

,id,label,tweet,cleaned
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation


In [22]:
x = data.drop(['id','tweet','label'],axis=1)
y = data['label']

In [23]:
x.cleaned

0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i can't use cause th...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31957                             ate  isz that youuu?ai  
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960     #sikh #temple vandalised in in #calgary, #wso...
31961                          thank you  for you follow  
Name: cleaned, Length: 31962, dtype: object

In [30]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [25]:
train_x.shape

(25569, 1)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [100]:
tfvectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer,ngram_range=(1,2))

In [117]:
train_tf = tfvectorizer.fit(train_x['cleaned']).transform(train_x['cleaned'])

In [118]:
train_tf

<25569x33855 sparse matrix of type '<class 'numpy.float64'>'
	with 179657 stored elements in Compressed Sparse Row format>

In [161]:
classifier = RandomForestClassifier()

In [162]:
classifier.fit(train_tf, train_y)

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [121]:
te = tfvectorizer.transform(test_x['cleaned'])

In [163]:
pred = classifier.predict(te)

In [164]:
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [165]:
f1_score(test_y, pred)

0.6127527216174183

In [166]:
roc_auc_score(test_y, pred)

0.7352466725326081